In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/'My Drive'/MusicTransformer

/content/drive/My Drive/MusicTransformer


In [3]:
ls

bin/      model.py   preprocess.py     seq_test.py  train.ipynb
custom/   out.mid    __pycache__/      sequence.py  train.py
data.py   out.midi   README.md         testdata/    utils.py
dataset/  params.py  requirements.txt  test.mid


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 2.2MB 14.4MB/s 
     |████████████████████████████████| 2.8MB 47.6MB/s 
     |████████████████████████████████| 61kB 27.6MB/s 
     |████████████████████████████████| 17.3MB 40.2MB/s 
     |████████████████████████████████| 3.0MB 31.7MB/s 
     |████████████████████████████████| 332.1MB 53kB/s 
     |████████████████████████████████| 419kB 45.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/51/7d/db/0e38d2ec57843d00cc39f8df3686984ccec689694f7bc78a38
  Stored in directory: /root/.cache/pip/wheels/6c/c8/80/32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
Successfully built config progress
ERROR: magenta 0.3.19 has requirement mido==1.2.6, but you'll have mido 1.2.9 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9

In [5]:
from model import MusicTransformer
from custom.layers import *
from custom import callback
from tensorflow.python import keras
import params as par
from tensorflow.python import enable_eager_execution
from tensorflow.python.keras.optimizer_v2.adam import Adam
from data import Data
import utils
tf.executing_eagerly()

True

In [0]:
l_r = 0.0024 #@param {type:"slider", min:0, max:0.1, step:0.0001}
batch_size = 3 #@param {type:"slider", min:1, max:100, step:1}
pickle_dir = "processed/" #@param {type:"string"}
max_seq = 2048 #@param {type:"slider", min:1, max:3000, step:1}
epochs = 1416 #@param {type:"slider", min:1, max:10000, step:1}
model_save_path = "bin/models" #@param {type:"string"}

In [0]:
dataset = Data('dataset/processed/')
opt = Adam(par.l_r)
mt = MusicTransformer(
    embedding_dim=par.embedding_dim, 
    vocab_size=par.vocab_size, 
    num_layer=6, 
    max_seq=2048,
    debug=False)
mt.compile(optimizer=opt, loss=callback.TransformerLoss())

# Train Model

In [11]:
    for e in range(epochs):
        for b in range(len(dataset.files) // batch_size):
            batch_x, batch_y = dataset.seq2seq_batch(batch_size, par.max_seq)
            result_metrics = mt.train_on_batch(batch_x, batch_y)
            if b % 100 == 0:
              print('===========================================\n')
              print('Loss: {:6.6}, Accuracy: {:3.2}'.format(result_metrics[0], result_metrics[1]))
              mt.save('mt-2048-h4-dim256.h5', )
              
            


Loss: 5.81973, Accuracy: 9.2


NotImplementedError: ignored

# Generate Model

In [0]:
def fill_with_placeholder(prev_data: list, max_len: int, max_val: int=239):
    placeholder = [max_val for _ in range(max_len - len(prev_data))]
    return placeholder+prev_data

In [0]:
for i in range(max_seq):
    